In [1]:
# Indices

import pandas as pd
import numpy as np

year = range(2000,2035)

fuel = [     
    "sähkö",
    "lämpö ja kylmä",
    "maakaasu",
    "nestekaasu",
    "lämmitysöljy",
    "diesel",
    "bensiini",
    "ruskohiili",
    "hiili",
    "muut fossiiliset (sis. turve)",
    "kasviöljy",
    "biopolttoaine",
    "muu biomassa",
    "aurinkoterminen",
    "maalämpö",
    "tuulivoima",
    "vesivoima",
    "aurinkosähkö"
]

emission = [
    "hiilidioksidi",
    "metaani",
    "typpidioksidi",
    "typpioksiduuli",
    "hiilimonoksidi",
    "hiukkaset",
    "rikkidioksidi",
    "typenoksidit",
    "fluorihiilivedyt",
    "perfluorihiilivedyt",
    "rikkiheksafluoridi",
    "typpitrifluoridi",
    "F-kaasut",
    "pienhiukkaspäästö",
    "kasvihuonekaasupäästö"
]

sector = [
    "rakennukset",
    "kunnalliset rakennukset, laitteistot/tilat",
    "ei-kunnalliset rakennukset, laitteistot/tilat",
    "asuinrakennukset",
    "julkinen valaistus",
    "teollisuus, ei-ETS",
    "teollisuus, ETS (ei suositella)",
    "kuljetus",
    "kunnalliskalusto",
    "julkinen liikenne",
    "yksityinen ja kaupallinen liikenne",
    "energia",
    "paikallinen sähköntuotanto",
    "paikallinen lämmön ja kylmän tuotanto",
    "vesi",
    "jätteet",
    "maankäytön suunnittelu",
    "maatalous ja metsänhoito (sis. kalanjalostamot)",
    "ympäristö ja biodiversiteetti",
    "terveys",
    "pelastuspalvelut ja hätäpalvelut",
    "turismi",
    "muu sektori",
    "sopeutuminen"
]

subsector_of = [0,0,0,0,0,0,0,7,7,7,7,11,11,11,14,15,16,17,18,19,20,21,22,23]

treatment = ['solid waste disposal', 'composting', 'digestion', 'incineration', 'waste water treatment']

source = ['in - in', 'in - out', 'out - in']

building = ['residential', 'commercial and institutional', 'industry and construction', 'energy industries']

final_use = ['district', 'electricity', 'gas', 'geothermal', 'oil']

energy_class = ['old', 'new']

mode = ['walking', 'cycling', 'private car', 'bus', 'truck', 'van', 'tram', 'metro', 'train', 'flight', 'boat', 'machinery']

mode0 = ['road','rail','marine','aviation','off-road']

animal = ['cow', 'pig', 'sheep']

landuse = ['forest', 'field', 'turf field', 'peatland']

scope = ['1','2','3']

gas = ['CO2', 'CH4'] # Add PM2.5 and other gases relevant for other impacts

# https://ghgprotocol.org/sites/default/files/standards/GHGP_GPC_0.pdf Figure 2
gpc = pd.read_csv("gpc_classification.csv")[['gpc','basic','scope','name']].set_index(['gpc','basic','scope'])

def global_warming_potential():
    df = pd.DataFrame({'gas':gas, 'result':[1,10]})
    return df.set_index('gas')

# A test function to replace groupby for making sure that Iter column is not dropped even if it is not explicitly mentioned.
# This can be used if we end up using probabilistic variables.
# df is the variable dataframe and keep is a list of index column names to keep in the outcome.

def grupby(df, keep):
    out = list(set(df.columns) & set(keep+['Iter']))
    out = df.groupby(out)
    return out

#display(grupby(ghg_emissions_from_heating_of_stationary_sources().assign(Iter = 1), ['scope']).sum().reset_index())

In [2]:
################ Heating of buildings

## Energy use

def floor_area_of_buildings():
    df = pd.DataFrame({
        'building':building*len(final_use)*len(energy_class),
        'final_use':np.repeat(final_use*len(energy_class), len(building)),
        'energy_class':np.repeat(energy_class,len(building)*len(final_use)),
        'result':range(len(building)*len(final_use)*len(energy_class))}) # m²
    return df.set_index(['building','final_use','energy_class'])

def energy_efficiency_of_buildings():
    df = pd.DataFrame({
        'building':building*len(energy_class),
        'energy_class':np.repeat(energy_class, len(building)),
        'result':[4,5,6,7,1,2,3,4] # kWh/m²/a
    })
    return df.set_index(['building','energy_class'])

def energy_use_from_heating_of_buildings(
    p1 = floor_area_of_buildings(),
    p2 = energy_efficiency_of_buildings()):

    df = p1 * p2 * 1E-6 # kWh/a --> GWh/a
    return df

def emission_factors_of_energy():
    df = pd.DataFrame({
        'final_use':final_use*len(gas),
        'scope':np.repeat(['2','2','2','2','1'],len(gas)),
        'gas':np.repeat(gas,len(final_use)),
        'result':range(len(gas)*len(final_use)) # g/kWh = ton/GWh
    })
    return df.set_index(['final_use','scope','gas'])

def loss_from_the_grid(): # Loss in the system, given as a ratio to the FINAL energy consumption
    df = pd.DataFrame({
        'final_use':final_use,
        'scope':'3',
        'result':[0.05, 0.04, 0.1, 0.04, 0.02]}) # % # ALSO CONTAINS SYSTEM INEFFICIENCIES (hyötysuhde) BUT THIS DEPENDS ON Production type
    return df.set_index(['final_use','scope'])

def emissions_from_heating_of_stationary_sources(
    p1 = energy_use_from_heating_of_buildings(),
    p2 = emission_factors_of_energy(),
    p3 = loss_from_the_grid()):

    df = p1 * p2 # ton/a

    loss = df.reset_index('scope').drop('scope',1) * p3

    df = pd.concat([df.reset_index(), loss.reset_index()])
    df = df.groupby(['building','final_use','gas','scope']).sum()

    return df

def ghg_emissions_from_heating_of_stationary_sources(
    p1 = emissions_from_heating_of_stationary_sources(),
    p2 = global_warming_potential()):

    df = p1 * p2 * 1E-3 # ton/a --> kton/a (CO2e)
    df = df.reset_index('gas').drop('gas',1).groupby(['building','final_use','scope']).sum()

    return df

#display(ghg_emissions_from_heating_of_stationary_sources())

In [3]:
######### Consumer electricity

def specific_electricity_consumption():
    df = pd.DataFrame({
        'building':building,
        'result':[150, 100, 120, 100] # kWh/m²/a
    })
    return df.set_index('building')

def energy_use_from_consumer_electricity(
    p1 = floor_area_of_buildings(),
    p2 = specific_electricity_consumption()):
    
    p1 = p1.groupby('building').sum()
    p1['final_use'] = 'electricity'
    p1 = p1.set_index('final_use', append = True)
    df = p1 * p2 * 1E-6 # kWh/a --> GWh/a
    df = df.groupby(['building','final_use']).sum()

    return df

def emissions_from_consumer_electricity(
    p1 = energy_use_from_consumer_electricity(),
    p2 = emission_factors_of_energy(),
    p3 = loss_from_the_grid()):

    df = p1 * p2 # ton/a

    loss = df.reset_index('scope').drop('scope',1) * p3

    df = pd.concat([df.reset_index(), loss.reset_index()]).dropna() # non-electricity final uses dropped

    df = df.groupby(['building','final_use','gas','scope']).sum()
    
    return df

def ghg_emissions_from_consumer_electricity(
    p1 = emissions_from_consumer_electricity(),
    p2 = global_warming_potential()):

    df = p1 * p2 * 1E-3 # ton/a --> kton/a (CO2e)
    df = df.groupby(['building','final_use','scope']).sum()

    return df

#display(ghg_emissions_from_consumer_electricity())

In [4]:
# Missing stationary sources:

# I.5.X Agriculture, forestry, and fishing activities
# I.6.X Non-specified sources
# I.7.X Fugitive emissions from mining, processing, storage, and transportation of coal
# I.8.X Fugitive emissions from oil and natural gas systems

In [5]:
################# Transport

def transport_activity_of_people():
    df = pd.DataFrame({
        'transboundary':source,
        'result':[290,50,10] # trips per day
    })
    
    return df.set_index(['transboundary'])

def average_trip_length():
    df = pd.DataFrame({
        'transboundary':source*len(mode),
        'mode':np.repeat(mode, len(source)),
        'result':range(len(mode)*len(source)) # km/trip
    })
    
    return df.set_index(['transboundary','mode'])

def modal_share_of_transport():
    df = pd.DataFrame({
        'transboundary':source*len(mode),
        'mode':np.repeat(mode, len(source)),
        'result':range(len(mode)*len(source)) # fraction
    })
    
    return df.set_index(['transboundary','mode'])

def energy_intensity_of_transport_modes():
    df = pd.DataFrame({
        'mode':mode,
        'fuel':['electricity', 'electricity', 'gasoline', 'diesel', 'diesel', 'diesel', 'electricity', 'electricity', 'electricity','kerosene','light oil','diesel'],
        'result':range(len(mode)) # MJ/km
    })
    
    return df.set_index(['mode','fuel'])

def emission_factors_of_transport_modes():
    df = pd.DataFrame({
        'fuel':['electricity', 'gasoline', 'diesel','kerosene','light oil']*len(gas),
        'scope':['2','1','1','1','1']*len(gas),
        'gas':np.repeat(gas,5),
        'result':[1, 5, 9, 2, 6, 7, 2, 5, 3,7] # g/MJ = ton/TJ
    })
    
    return df.set_index(['fuel','scope','gas'])

def energy_use_from_transportation( # NOTE! Energy losses are not considered (yet)
    p1 = transport_activity_of_people(),
    p2 = modal_share_of_transport(),
    p3 = energy_intensity_of_transport_modes()):
    
    df = p1 * p2 * p3 * 1E-6 * 365.25 # MJ/d --> TJ/a
    df = df.groupby(['transboundary','fuel','mode']).sum()
    
    return df

def emissions_from_transportation( # NOTE! Energy losses are not considered (yet)
    p1 = energy_use_from_transportation(),
    p2 = emission_factors_of_transport_modes()):
    
    df = p1 * p2 # ton/a
    df = df.groupby(['transboundary','fuel','scope','gas','mode']).sum()
    
    return df

def ghg_emissions_from_transportation(
    p1 = emissions_from_transportation(),
    p2 = global_warming_potential()):
    
    df = p1 * p2 * 1E-3 # ton/a --> kton/a (CO2e)
    df = df.groupby(['transboundary','fuel','scope','mode']).sum()
    
    return df

#display(ghg_emissions_from_transportation())

In [6]:
####### Waste

def amount_of_waste():
    df = pd.DataFrame({
        'treatment':treatment*len(source),
        'source':np.repeat(source, len(treatment)),
        'result':range(len(source)*len(treatment)) # ton/a
    })
    return df.set_index(['treatment','source'])

def emission_factors_of_waste():
    df = pd.DataFrame({
        'treatment':treatment*len(gas),
        'gas':np.repeat(gas, len(treatment)),
        'result':np.repeat([1,2], len(treatment)) # kg/ton
    })
    return df.set_index(['treatment','gas'])

def emissions_from_waste(
    p1 = amount_of_waste(),
    p2 = emission_factors_of_waste()):
    
    df = p1 * p2 * 1E-3 # kg/a --> ton/a
    df = df.groupby(['treatment','gas','source']).sum() # redundant because there are currently no more indices
    
    return df

def ghg_emissions_from_waste(
    p1 = emissions_from_waste(),
    p2 = global_warming_potential()):
 
    df = p1 * p2 * 1E-3 # ton/a --> kton/a (CO2e)
    df = df.groupby(['treatment','source']).sum()
    
    return df

#display(ghg_emissions_from_waste())

In [7]:
################## AFOLU

def number_of_farm_animals():
    df = pd.DataFrame({
        'animal':animal,
        'result':[10000, 20000, 30000] # number
    })
    
    return df.set_index('animal')

def land_area():
    df = pd.DataFrame({
        'landuse':landuse,
        'result':[40000, 50000, 60000, 70000] # ha
    })
    
    return df.set_index('landuse')

def emission_factors_of_afolu():
    df = pd.DataFrame({
        'source':['livestock']*len(animal)+['land']*len(landuse)+['aggregate']*len(landuse),
        'animal':animal+[None]*len(landuse)*2,
        'landuse':[None]*len(animal)+landuse*2,
        'gas':['CO2']*(len(animal)+len(landuse))+['CH4']*len(landuse),
        'result':[2,3,4,5,6,7,8,9,10,11,12] # g/animal/a or g/ha/a
    })
    return df.set_index(['source','animal','landuse','gas'])

def emissions_from_afolu(
    p1 = number_of_farm_animals(),
    p2 = land_area(),
    p3 = emission_factors_of_afolu()):
    
    df = p1 * p3 * 1E-6 # g/a --> ton/a
    df = df.groupby(['source','gas']).sum()
    
    df2 = p2 * p3 * 1E-6 # g/a --> ton/a
    df2 = df2.groupby(['source','gas']).sum()
    
    df = pd.concat([df, df2])   
    
    return df

def ghg_emissions_from_afolu(
    p1 = emissions_from_afolu(),
    p2 = global_warming_potential()):
    
    df = p1 * p2 * 1E-3 # ton/a --> kton/a (CO2e)
    df = df.groupby(['source']).sum()
    
    return df

#display(ghg_emissions_from_afolu())

def why_does_field_appear_in_livestock( ## WTF?!?
    p1 = land_area(),
    p2 = emission_factors_of_afolu()):
    
    df = p1 * p2
    return df

display(emission_factors_of_afolu())
display(land_area())
display(why_does_field_appear_in_livestock())


result
source    animal landuse    gas        
livestock cow    NaN        CO2       2
          pig    NaN        CO2       3
          sheep  NaN        CO2       4
land      NaN    forest     CO2       5
                 field      CO2       6
                 turf field CO2       7
                 peatland   CO2       8
aggregate NaN    forest     CH4       9
                 field      CH4      10
                 turf field CH4      11
                 peatland   CH4      12

,result
landuse,
forest,40000
field,50000
turf field,60000
peatland,70000


result
source    animal landuse    gas        
livestock cow    field      CO2  100000
          pig    field      CO2  150000
          sheep  field      CO2  200000
land      NaN    forest     CO2  200000
                 field      CO2  300000
                 turf field CO2  420000
                 peatland   CO2  560000
aggregate NaN    forest     CH4  360000
                 field      CH4  500000
                 turf field CH4  660000
                 peatland   CH4  840000

In [8]:
############## IPPU

def emissions_from_ippu():
    df = pd.DataFrame({
        'source':['industrial processes','product use']*len(gas),
        'gas':np.repeat(gas,2),
        'result':[100, 20, 5, 8] # ton/a
    })
    
    return df.set_index(['source','gas'])

def ghg_emissions_from_ippu(
    p1 = emissions_from_ippu(),
    p2 = global_warming_potential()):

    df = p1 * p2 * 1E-3 # ton/a --> kton/a (CO2e)
    
    return df

#display(ghg_emissions_from_ippu())

In [9]:
##################### Other scope 3

def ghg_emissions_from_other_scope3():
    df = pd.DataFrame({
        'source':['other scope3'],
        'result':[20] # kton/a (CO2e)
    })
    
    return df.set_index(['source'])

#display(ghg_emissions_from_other_scope3())

In [10]:
##### Total GHG emissions from the municipality using GPC classification (compatible with basic/basic+ classification)
    
def total_ghg_emissions_from_the_municipality( # kton/a (CO2e)
    p1 = ghg_emissions_from_heating_of_stationary_sources().reset_index(),
    p2 = ghg_emissions_from_consumer_electricity().reset_index(),
    p3 = ghg_emissions_from_transportation(),
    p4 = ghg_emissions_from_waste().reset_index(),
    p5 = ghg_emissions_from_ippu().reset_index(),
    p6 = ghg_emissions_from_afolu().reset_index(),
    p7 = ghg_emissions_from_other_scope3().reset_index()):
    
    tmp = pd.DataFrame({
        'building':np.repeat(building, 3),
        'scope':['1','2','3']*len(building),
        'gpc':['I.1.1','I.1.2','I.1.3','I.2.1','I.2.2','I.2.3','I.3.1','I.3.2','I.3.3','I.4.1','I.4.2','I.4.3']
    })
    
    tmp = pd.concat([p1, p2]).merge(tmp)
    
    df = tmp.groupby(['gpc']).sum()

    tmp = p3 * pd.DataFrame({ # Half of the emission is allocated to starting point and half to destination
        'transboundary':source,
        'result':[1, 0.5, 0.5]
    }).set_index('transboundary')
    
    tmp = tmp.reset_index().merge(pd.DataFrame({
        'mode':mode,
        'mode0':['road']*6 + ['rail']*3 + ['marine'] + ['aviation'] + ['off-road']
    })).merge(pd.DataFrame({
        'scope':scope*len(mode0),
        'mode0':np.repeat(mode0, len(scope)),
        'gpc':['II.1.1','II.1.2','II.1.3','II.2.1','II.2.2','II.2.3','II.3.1','II.3.2','II.3.3','II.4.1','II.4.2','II.4.3','II.5.1','II.5.2','II.5.3']
    }))

    tmp = tmp.groupby(['gpc']).sum()#['result'] # kton/a (CO2e)

    df = pd.concat([df, tmp])

    tmp = p4.merge(pd.DataFrame({
        'treatment':np.repeat(treatment, len(source)),
        'source':source*len(treatment),
        'gpc':['III.1.1','III.1.2','III.1.3','III.2.1','III.2.2','III.2.3','III.2.1','III.2.2','III.2.3','III.3.1','III.3.2','III.3.3','III.4.1','III.4.2','III.4.3']
    }))
    tmp = tmp.groupby('gpc').sum()

    df = pd.concat([df, tmp])

    tmp = p5.merge(pd.DataFrame({
        'source':['industrial processes','product use'],
        'gpc':['IV.1','IV.2']
    }))
    tmp = tmp.groupby('gpc').sum()

    df = pd.concat([df, tmp])

    tmp = p6.merge(pd.DataFrame({
        'source':['livestock','land','aggregate'],
        'gpc':['V.1','V.2','V.3']
    }))
    tmp = tmp.groupby('gpc').sum()

    df = pd.concat([df, tmp])

    tmp = p7.merge(pd.DataFrame({
        'source':['other scope3'],
        'gpc':['VI.1']
    }))
    tmp = tmp.groupby('gpc').sum()

    df = pd.concat([df, tmp])
    df = df.reset_index().merge(gpc.reset_index()).set_index(['name','gpc','basic','scope'])

    return df

display(total_ghg_emissions_from_the_municipality())

,,,,result
name,gpc,basic,scope,
residential fuel,I.1.1,basic,1,0.000015
residential grid,I.1.2,basic,2,0.001655
residential losses,I.1.3,basic+,3,0.000067
commercial fuel,I.2.1,basic,1,0.000025
commercial grid,I.2.2,basic,2,0.001174
commercial losses,I.2.3,basic+,3,0.000048
manufacturing fuel,I.3.1,basic,1,0.000035
manufacturing grid,I.3.2,basic,2,0.001487
manufacturing losses,I.3.3,basic+,3,0.000061


In [11]:
####### Total energy use and GHG emissions using SECAP reporting

### NOTE! The end result should be more specific to be useful for SECAP

def energy_use_of_the_municipality( # GWh/a
    p1 = energy_use_from_heating_of_buildings().reset_index(),
    p2 = energy_use_from_consumer_electricity().reset_index(),
    p3 = energy_use_from_transportation().reset_index()):
    
    p1['sector'] = 'heating'
    p2['sector'] = 'electricity'
    p3['sector'] = 'transportation'
    
    df = pd.concat([p1, p2])
    p3['final_use'] = p3['fuel']
    p3['result'] / 3.6 # TJ/a --> GWh/a
    df = pd.concat([df[['sector','final_use','result']], p3[['sector','final_use','result']]])
    df = df.groupby(['sector','final_use']).sum()
    
    return df

#display(energy_use_of_the_municipality())